In [ ]:
!pip install keras
!pip install keras==3.05
!pip install deepface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0


In [ ]:
import numpy as np
import cv2 as cv
from PIL import Image
import tensorflow as tf
# from keras.vggface.vggface import VGGFACE
# from keras_vggface.utils import preprocess_input
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from deepface import DeepFace

25-07-10 13:01:24 - Directory /root/.deepface has been created
25-07-10 13:01:24 - Directory /root/.deepface/weights has been created


In [ ]:
face_cascade=cv.CascadeClassifier(cv.data.haarcascades + 'haarcascades_frontalface_default')

project
# "Office Access" - A Simple Employee Verification System

    Case Study: A small tech company wants to build a prototype for a new office access system. When an employee arrives at the door, a camera takes their picture. The system should verify if the person is a registered employee before unlocking the door.

    Your Goal: Build the core logic for this system using the deepface library.

    The Challenge:

        Create Your Database: Gather 2-3 photos each of 3-4 different "employees" and organize them into folders.

        Build the System: Adapt the instructor's new code to build your employee face database. Use the get_embedding function provided.

        Create the Verifier: Write a function verify_employee(image_path, database) that returns the name of the employee if they are found within the distance threshold, and "Access Denied" if they are not

In [ ]:
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
employee_database='/content/drive/MyDrive/Photos-1-001'

In [ ]:
import os

# Base directory
base_dir = "employee_database"

# Employee names
employees = ["Gloria", "VK"]

# Create folders
for name in employees:
    folder_path = os.path.join(base_dir, name)
    os.makedirs(folder_path, exist_ok=True)
    print(f"📁 Created: {folder_path}")

📁 Created: employee_database/Gloria
📁 Created: employee_database/VK


In [ ]:
# Save image example
from PIL import Image
img = Image.open("/content/drive/MyDrive/Photos-1-001/IMG_20250128_145226.jpg")
img.save(os.path.join("employee_database", "Gloria", "Gloria_1.jpg"))

In [ ]:
employee_database = "employee_database"
employees = ["Gloria", "VK"]

import os

for name in employees:
    person_folder = os.path.join(employee_database, name)
    os.makedirs(person_folder, exist_ok=True)
    print(f"📁 Created folder for {name}: {person_folder}")

📁 Created folder for Gloria: employee_database/Gloria
📁 Created folder for VK: employee_database/VK


In [ ]:
person_folder = os.path.join(employee_database, employees)
os.makedirs(person_folder, exist_ok=True)
filename = f"{uuid.uuid4().hex}.jpg"
save_path = os.path.join(person_folder, filename)
image.save(save_path)

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'list'

In [ ]:
for root, dirs, files in os.walk("employee_database"):
    level = root.replace("employee_database", "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    for file in files:
        print(f"{indent}    {file}")

employee_database/
    VK/
    Gloria/
        Gloria_1.jpg


In [ ]:
employee_dir = "employee_database"
embedding_db = {}

def get_embedding(image_path):
    try:
        emb_obj = DeepFace.represent(img_path=image_path, model_name='Facenet', enforce_detection=True)
        return emb_obj[0]['embedding']
    except ValueError:
        return None

def build_embedding_database():
    global embedding_db
    embedding_db = {}
    for employee in os.listdir(employee_dir):
        person_folder = os.path.join(employee_dir, employee)
        if os.path.isdir(person_folder):
            embeddings = []
            for img_file in os.listdir(person_folder):
                img_path = os.path.join(person_folder, img_file)
                emb = get_embedding(img_path)
                if emb is not None:
                    embeddings.append(emb)
            if embeddings:
                embedding_db[employee] = np.mean(embeddings, axis=0)
                print(f"✅ Registered: {employee}")


In [ ]:
from scipy.spatial.distance import cosine

def verify_employee(image_path, database, threshold=0.4):
    unknown_embedding = get_embedding(image_path)
    if unknown_embedding is None:
        return "No face detected — try again"

    min_dist = float('inf')
    match_name = "Access Denied"

    for name, registered_emb in database.items():
        dist = cosine(registered_emb, unknown_embedding)
        if dist < min_dist:
            min_dist = dist
            match_name = name

    return f"✅ Welcome {match_name}" if min_dist <= threshold else "🚫 Access Denied"

In [ ]:
build_embedding_database()
result = verify_employee("live_photo.jpg", embedding_db)
print(result)